# Reinforcement Learning
- Following [this](https://www.youtube.com/watch?v=Mut_u40Sqz4) tutorial

## Import dependencies

In [ ]:
!pip install stable-baselines3[extra]

In [ ]:
import gym
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Load Environment

In [ ]:
environment_name = 'CartPole-v0'
env = gym.make(environment_name)

### Test the environment

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

## Train a RL Model

In [ ]:
log_path = os.path.join("../output", "training", "logs")

In [ ]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000)

## Save and Reload Model

In [ ]:
ppo_path = os.path.join("../output", "training", "models", "ppo_model_partpole")

In [ ]:
model.save(ppo_path)

In [ ]:
del model

In [ ]:
model = PPO.load(ppo_path, env=env)

## Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

## Use model to predict action

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action, _ = model.predict(obs)
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Episode: {episode}, Score: {score}")
env.close()

## Viewing logs in Tensorboard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir={log_path}

## Adding Callbacks (auto save)

In [ ]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [ ]:
save_path = os.path.join("../output", "training", "saved models")

In [ ]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(
    env,
    callback_on_new_best=stop_callback,
    eval_freq=10000,
    best_model_save_path=save_path,
    verbose=1)


In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

## Change Policies

In [ ]:
net_arch = [dict(
    pi=[128, 128, 128, 128],
    vf=[128, 128, 128, 128])]

In [ ]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path,
            policy_kwargs={'net_arch': net_arch})

In [ ]:
model.learn(total_timesteps=20000, callback=eval_callback)

## Using alternative training algorithm (`DQN`)

In [ ]:
from stable_baselines3 import DQN

In [ ]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

In [ ]:
model.learn(total_timesteps=200000, callback=eval_callback)